In [2]:
pip install psycopg2

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import psycopg2
import pandas as pd
import csv

In [4]:
# Create a database if not existing
# try:
#     cur.execute("create database Bike_Store")
# except psycopg2.Error as e:
#     print(e)

In [27]:
def create_database():

    #create database connection
    conn = psycopg2.connect("""host=localhost 
                            dbname=postgres
                            user=postgres 
                            password=root
                            """)
    #autocommit
    conn.set_session(autocommit=True)
    #create cursor object
    cur=conn.cursor()

    # Create bike_store database
    cur.execute("DROP DATABASE IF EXISTS Bike_Store")
    cur.execute("CREATE DATABASE Bike_Store")

    # Close default db connection
    cur.close()
    conn.close()

    #Connect to Bike_Store database
    conn = psycopg2.connect("host=localhost dbname=bike_store user=postgres password=root")
    cur = conn.cursor()
     
    return cur, conn


In [6]:
#load data to dataframes for manipulation before uploading
customers = pd.read_csv('data/customers.csv')
orders = pd.read_csv('data/orders.csv')
order_items = pd.read_csv('data/order_items.csv')
brands = pd.read_csv('data/brands.csv')
categories = pd.read_csv('data/categories.csv')
products = pd.read_csv('data/products.csv')
staffs = pd.read_csv('data/staffs.csv')
stocks = pd.read_csv('data/stocks.csv')
stores = pd.read_csv('data/stores.csv')

In [7]:
customers.head()

,customer_id,first_name,last_name,phone,email,street,city,state,zip_code
0,1,Debra,Burks,NaN,debra.burks@yahoo.com,9273 Thorne Ave.,Orchard Park,NY,14127
1,2,Kasha,Todd,NaN,kasha.todd@yahoo.com,910 Vine Street,Campbell,CA,95008
2,3,Tameka,Fisher,NaN,tameka.fisher@aol.com,769C Honey Creek St.,Redondo Beach,CA,90278
3,4,Daryl,Spence,NaN,daryl.spence@aol.com,988 Pearl Lane,Uniondale,NY,11553
4,5,Charolette,Rice,(916) 381-6003,charolette.rice@msn.com,107 River Dr.,Sacramento,CA,95820


In [8]:
# remove PII
customers.drop(['phone','email'], axis=1, inplace=True)
customers.head()

,customer_id,first_name,last_name,street,city,state,zip_code
0,1,Debra,Burks,9273 Thorne Ave.,Orchard Park,NY,14127
1,2,Kasha,Todd,910 Vine Street,Campbell,CA,95008
2,3,Tameka,Fisher,769C Honey Creek St.,Redondo Beach,CA,90278
3,4,Daryl,Spence,988 Pearl Lane,Uniondale,NY,11553
4,5,Charolette,Rice,107 River Dr.,Sacramento,CA,95820


In [30]:
cur, conn = create_database()

In [36]:
# """ """ are sused for creating multi-line strings or docstring
customer_table = ("""CREATE TABLE IF NOT EXISTS Customers(customer_id INT PRIMARY KEY, 
                  first_name VARCHAR (255) NOT NULL, 
                  last_name VARCHAR (255) NOT NULL, 
                  street VARCHAR (255),
                  city VARCHAR (50),
                  state VARCHAR (25),
                  zip_code VARCHAR (5)
                  );""")

In [ ]:
order_items_table =("""CREATE TABLE IF NOT EXISTS Order_items(
                    order_id INT,
	                item_id INT,
	                product_id INT NOT NULL,
	                quantity INT NOT NULL,
	                list_price DECIMAL (10, 2) NOT NULL,
	                discount DECIMAL (4, 2) NOT NULL DEFAULT 0,
	                PRIMARY KEY (order_id, item_id),
	                FOREIGN KEY (order_id) REFERENCES sales.orders (order_id) ON DELETE CASCADE ON UPDATE CASCADE,
	                FOREIGN KEY (product_id) REFERENCES production.products (product_id) ON DELETE CASCADE ON UPDATE CASCADE
                    );
                    """)

In [ ]:
orders_table =("""CREATE TABLE IF NOT EXISTS Orders(
                order_id INT IDENTITY (1, 1) PRIMARY KEY,
	            customer_id INT,
	            order_status tinyint NOT NULL,
	            -- Order status: 1 = Pending; 2 = Processing; 3 = Rejected; 4 = Completed
	            order_date DATE NOT NULL,
	            required_date DATE NOT NULL,
	            shipped_date DATE,
	            store_id INT NOT NULL,
	            staff_id INT NOT NULL,
	            FOREIGN KEY (customer_id) REFERENCES sales.customers (customer_id) ON DELETE CASCADE ON UPDATE CASCADE,
	            FOREIGN KEY (store_id) REFERENCES sales.stores (store_id) ON DELETE CASCADE ON UPDATE CASCADE,
	            FOREIGN KEY (staff_id) REFERENCES sales.staffs (staff_id) ON DELETE NO ACTION ON UPDATE NO ACTION
                );
                """)

In [37]:
cur.execute(customer_table)
conn.commit()